# AQI Classifier

## Importing Scraped Data

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

'''
#If dataset is in google drive, it can be imported from there as well 
from google.colab import drive
drive.mount('/content/drive') 

# Define directories for training and validation data
train_dir = '/content/drive/MyDrive/AQI_Estimator/Training'
validation_dir = '/content/drive/MyDrive/AQI_Estimator/Validation'
'''

train_dir= '/AQI_Estimator_Dataset/Training'
validation_dir = '/AQI_Estimator_Dataset/Validation'